In [1]:
import numpy as np
import pandas as pd
import datetime
import pytz
import requests
import wordle_cheater as wrdlc
from IPython import display

___
### Basic usage of the ```wordle_cheater```

Create an instance of the class ```wordl``` in the```wordle_cheater``` module to help solve for the wordle solution **whine**.

In [2]:
# instantiate the wordl class
wdl = wrdlc.wordl()
# get todays word
td_word = wdl.todays_word

___
### Add the word *alert* to the ```wdl``` instance.

   * Each word that you add to the ```wordl``` instance requires an array of 5 integers between 1 and 3 (inclusive), for each of the 5 letters of the word that you add.  This array is called the ```letter_status```. 
   * The ```letter_status``` array tells the ```wordl.try_it()``` method one of 3 things about the respective letter:
     1. The letter code is **1** for that letter if:
       * that letter is *in the solution word*, and 
       * the letter's position in the word is *the same as the letter's position in the solution word*, 
     2. The letter code is **2** for that letter if:
       * the letter is *in the solution word*, 
       * but *NOT in the same position as the solution word*
     3. The letter code is **3** for that letter if:
       * the letter is NOT at all in the solution word

In other words:
  * a status code of **1** is equal to the color **GREEN** in the Wordle Game, 
  * a status code of **2** is equal to the color **GOLD** in the Wordle Game, 
  * a status code of **3** is equal to the color **GREY** in the Wordle Game, 


#### For Example, when the solution word is **whine**:
In the word **alert**:
  1. The letter **a** is not in the word **whine**, so it's letter code = 3
  2. The letter **l** is not in the word **whine**, so it's letter code = 3
  3. The letter **e** is in the word **whine**, but NOT is the correct position, so it's letter code = 2
  4. The letter **r** is not in the word **whine**, so it's letter code = 3
  4. The letter **t** is not in the word **whine**, so it's letter code = 3

So, for the word **alert**, the letter_status array is:
  * ```[3,3,2,3,3]```

In [3]:
# add the word 'alert' to the instance
wdl.add_word('alert',[3,3,2,3,3])
df = wrdlc.filter_words(wdl.try_it()) 
df.probability = df.probability.round(5)
display.display(
    df.style.set_caption(
        "Possible Words After adding 'alert' to the wdl instance"
    )
)

,word,count,probability
1,video,365410017,0.106700
2,phone,256643812,0.074940
3,index,242826246,0.070910
4,being,242783091,0.070900
5,women,242520455,0.070820
6,house,231310420,0.067550
7,since,214302926,0.062580
8,guide,213378807,0.062310
9,money,190205072,0.055540
10,movie,158421100,0.046260


___
### Add the word *noisy* to the ```wdl``` instance.

In the word **noisy**:
  1. The letter **n** is in the word **whine**, but NOT is the correct position, so it's letter code = 2
  2. The letter **o** is not in the word **whine**, so it's letter code = 3
  3. The letter **i** is in the word **whine**, AND it is in the correct position, so it's letter code = 1
  4. The letter **s** is not in the word **whine**, so it's letter code = 3
  4. The letter **t** is not in the word **whine**, so it's letter code = 3

So, for the word **noisy**, the letter_codes array is:
  * ```[2,3,1,3,3]```

In [4]:
wdl.add_word('noisy',[2,3,1,3,3])
df = wrdlc.filter_words(wdl.try_it()) 
df.probability = df.probability.round(5)
display.display(
    df.style.set_caption(
        "Possible Words After adding 'noisy' to the wdl instance"
    )
)

,word,count,probability
1,being,242783091,0.951520
2,knife,11297715,0.044280
3,whine,832743,0.003260
4,feign,116753,0.000460
5,deign,94806,0.000370
6,eking,26695,0.000100


### Since *being* is the most probable word, add it

In the word **being**:
  1. The letter **b** is not in the word **whine**, so it's letter code = 3
  2. The letter **e**  is in the word **whine**, BUT is is NOT in the correct position, so it's letter code = 2
  3. The letter **i** is in the word **whine**, AND it is in the correct position, so it's letter code = 1
  4. The letter **n** is in the word **whine**, AND it is in the correct position, so it's letter code = 1
  4. The letter **g** is not in the word **whine**, so it's letter code = 3

So, for the word **being**, the letter_codes array is:
  * ```[3,2,1,1,3]```

In [5]:
wdl.add_word('being',[3,2,1,1,3])
df = wrdlc.filter_words(wdl.try_it()) 
df.probability = df.probability.round(5)
display.display(
    df.style.set_caption(
        "Possible Words After adding 'being' to the wdl instance"
    )
)

,word,count,probability
1,whine,832743,1.000000


___
### The only word left is *whine*.
___

___
### Methods to create the letter_status lists automatically

The ```wordle_cheater.wordl.add_word(word,letter_status)``` has 2 arguments, ```word``` and ```letter_status```.  The ```letter_status``` argument is a list of integers between 1 and 3 inclusive.

In [6]:
def get_letter_status(your_word,the_word):
    nums =[]
    for i,l in enumerate(list(your_word)):
        if l == the_word[i]:
            nums.append(1)
        elif l in the_word:
            nums.append(2)
        else:
            nums.append(3)
    return nums    
        
def letter_list_to_str(letter_list):
    return ''.join(
        [str(v) for v in letter_list]
    )
def get_filter(your_word,the_word):
    letter_string = letter_list_to_str(get_letter_list(your_word,td_word))
    return your_word + "," + letter_string

### Below are methods that will find the solution to the the word, given the date, and your 2 first words.


In [7]:
# Create a wordle_cheater.wordl instance
wdl = wrdlc.wordl()
# Define your first 2 words
initial_words = ['alert','noisy']

# Define a month and a day
month = "Dec"
day = 1
month_day = month + " " + str(day) + " -"
df_wdlh = wdl.df_word_history[wdl.df_word_history['date']<=month_day]

# Get the first occurrence which is that day, or lower
df_wdlh = df_wdlh.iloc[0:1]

# Get that days solution
solution = df_wdlh.solution.iloc[0].lower()

# Loop on 5 times, either loading wdl instance with words from the
#    intial_list array, or by loading the most possible word that
#    the method wrdlc.filter_words returns (after there are no more intial_words)
# Also, save the list of possible words at after loading a new word.
list_df = []
words_used = []
for i in range(5):
    if i<len(initial_words):
        w = initial_words[i]
    else:
        # Get last DataFrame from list_df
        df_last = list_df[i-1]
        w = df_last.iloc[0].word
    words_used.append(w)
    letter_status = get_letter_status(w,solution)
    # Add a word to the wdl instance
    wdl.add_word(w,letter_status)
    wdl_possible_words_json = wdl.try_it() 
    # Append the DataFrame with the list of possible words, at this stage
    df = wrdlc.filter_words(wdl_possible_words_json).copy()
    list_df.append(df)
    if len(df)<=1:
        break



In [8]:
display.display(
    df_wdlh.style.set_caption(f'Solution for {month} {day}')
)

,date,number,solution
19,Dec 09,538,BRAID


In [9]:
display.display(f"words used were: {words_used}")

"words used were: ['alert', 'noisy', 'chair']"

In [10]:
for i,df in enumerate(list_df):
    display.display(
        df.style.set_caption(
            f"Possible Words after word: {words_used[i]}"
        )
    )

,word,count,probability
1,march,158281269,0.146089
2,major,122592252,0.113149
3,radio,113285624,0.104559
4,brand,82807581,0.076429
5,grand,56599468,0.052240
6,chair,40764005,0.037624
7,urban,38087999,0.035154
8,brain,33612784,0.031024
9,frank,31392284,0.028974
10,crazy,23032837,0.021259


,word,count,probability
1,chair,40764005,0.552444
2,rapid,19070672,0.258450
3,cigar,3993843,0.054126
4,rabbi,3488700,0.047280
5,circa,3475462,0.047100
6,braid,981945,0.013308
7,vicar,825292,0.011185
8,rabid,612676,0.008303
9,radii,575939,0.007805


,word,count,probability
1,braid,981945,1.000000


In [11]:
# # define your first 2 words
# initial_words = ['alert','noisy']
# # create initial filter string that gets used in the url for the 
# #   http call to the wordl server at http://127.0.0.1:8000/wordl
# initial_filters = ','.join(
#     np.array(
#         [
#             get_filter(word,get_filter(word,td_word))
#             for word in initial_words
#         ]
#     ).reshape(-1).tolist()
# )

# # The loop below will populate the string called filters.  
# #   This string will be added to during the for loop below.
# filters = initial_filters
# # results_list holds DataFrames that show possible words after 
# #   words are added to the filters string, and the server is called again
# results_list = []
# for _ in range(3):
#     url = f'http://127.0.0.1:8000/wordl?filters={filters}'
#     response = requests.get(url)
#     resp_dict = response.json()    
#     df_results = pd.DataFrame(resp_dict)
#     results_list.append(df_results.copy())
#     if len(df_results)<=1:
#         break
#     next_word = df_results.iloc[0].word
#     new_filter = get_filter(next_word,td_word)
#     filters = filters + "," + new_filter
    